In [9]:
import os
import math
import importlib
import pandas as pd
import matplotlib.pyplot as plt

import constants
import AtmosphereModel
import math_2d

# reload to pick up any edits
importlib.reload(constants)
importlib.reload(AtmosphereModel)
importlib.reload(math_2d)


<module 'math_2d' from 'c:\\Users\\Risha\\Desktop\\ReEntryAI\\StochasticEntrySim\\math_2d.py'>

In [10]:
def parse_input_file(input_path: str):
    """
    Parse a simple key = value input file into a dictionary.
    Lines starting with # or blank lines are ignored.
    Numeric values (including negatives and scientific notation)
    are converted to float; everything else is kept as string.
    """
    state_dict = {}

    with open(input_path, "r") as file:
        input_list = file.readlines()

    extract_pattern = r"^\s*([A-Za-z_]\w*)\s*=\s*([^#\n]+)"
    num_pattern = re.compile(r"^[+-]?\d+(\.\d+)?([eE][+-]?\d+)?$")

    for line in input_list:
        stripped = line.strip()
        if not stripped or stripped.startswith("#"):
            continue

        m = re.match(extract_pattern, line)
        if not m:
            continue

        key = m.group(1)
        raw_value = m.group(2).strip()

        if num_pattern.fullmatch(raw_value):
            try:
                value = float(raw_value)
                state_dict[key] = value
            except ValueError:
                state_dict[key] = raw_value
        else:
            state_dict[key] = raw_value

    return state_dict


## helpers to manage runs and save data

In [11]:
# -------- data folder / filenames --------
DATA_DIR = "data"
os.makedirs(DATA_DIR, exist_ok=True)

RUNS_CSV = os.path.join(DATA_DIR, "runs.csv")
TRAJ_CSV = os.path.join(DATA_DIR, "trajectories.csv")


def get_next_run_id() -> int:
    """Return the next integer run_id based on runs.csv."""
    if os.path.exists(RUNS_CSV):
        runs = pd.read_csv(RUNS_CSV)
        if runs.empty:
            return 1
        return int(runs["run_id"].max()) + 1
    else:
        return 1


def save_run(run_config: dict, history: dict) -> None:
    """
    Append one run to runs.csv and its time history to trajectories.csv.

    run_config: single-row metadata dict (includes run_id).
    history: dict of lists with time-series (step_index, t_s, h_m, V_mps, gamma_rad, s_m).
    """
    run_id = run_config["run_id"]

    # ---- save runs.csv (one row) ----
    runs_row = pd.DataFrame([run_config])
    if os.path.exists(RUNS_CSV):
        runs_row.to_csv(RUNS_CSV, mode="a", header=False, index=False)
    else:
        runs_row.to_csv(RUNS_CSV, index=False)

    # ---- save trajectories.csv (many rows) ----
    traj_df = pd.DataFrame({
        "run_id": run_id,
        "step_index": history["step_index"],
        "t_s": history["t_s"],
        "h_m": history["h_m"],
        "V_mps": history["V_mps"],
        "gamma_rad": history["gamma_rad"],
        "s_m": history["s_m"],
    })

    if os.path.exists(TRAJ_CSV):
        traj_df.to_csv(TRAJ_CSV, mode="a", header=False, index=False)
    else:
        traj_df.to_csv(TRAJ_CSV, index=False)


In [12]:
def run_planar_sim(init_state: dict, do_plots: bool = True):
    """
    init_state dict must contain:
        - "init_altitude_m"
        - "init_speed_mps"
        - "init_fpa_deg"
        - "sigma_const_deg"
    """

    case_name        = "LEO_entry_test_01"
    t0_s             = 0.0
    dt_s             = 0.25
    t_final_s        = 2000.0

    min_altitude_m   = 0.0
    min_speed_mps    = 10.0

    # unpack randomized initial state
    init_altitude_m = init_state["init_altitude_m"]
    init_speed_mps  = init_state["init_speed_mps"]
    init_fpa_deg    = init_state["init_fpa_deg"]
    sigma_const_deg = init_state["sigma_const_deg"]

    # vehicle/env params
    vehicle_params = {
        "mass_kg":     5000.0,
        "ref_area_m2": 10.0,
        "CL_const":    0.30,
        "CD_const":    1.00,
        "nose_radius_m": 1.0,
    }

    env_params = {
        "mu_m3s2":   3.986004418e14,
        "R_E_m":     6371000.0,
        "rho0_kgm3": 1.225,
        "H_m":       7200.0,
    }

    # --------------- initial state vector [h, V, gamma, s] ---------------
    y = [
        init_altitude_m,
        init_speed_mps,
        math.radians(init_fpa_deg),
        0.0,  # downrange
    ]

    t = t0_s
    sigma = math.radians(sigma_const_deg)

    # --------------- history buffers ---------------
    history = {
        "step_index": [],
        "t_s": [],
        "h_m": [],
        "V_mps": [],
        "gamma_rad": [],
        "s_m": [],
    }

    # --------------- run metadata (one row per run) ---------------
    run_id = get_next_run_id()

    run_config = {
        "run_id": run_id,
        "case_name": case_name,
        "t0_s": t0_s,
        "dt_s": dt_s,
        "t_final_s": t_final_s,
        "init_altitude_m": init_altitude_m,
        "init_speed_mps": init_speed_mps,
        "init_fpa_deg": init_fpa_deg,
        "sigma_const_deg": sigma_const_deg,
        "mass_kg": vehicle_params["mass_kg"],
        "ref_area_m2": vehicle_params["ref_area_m2"],
        "CL_const": vehicle_params["CL_const"],
        "CD_const": vehicle_params["CD_const"],
    }

    # --------------- time-stepping loop ---------------
    step = 0

    while t <= t_final_s:
        # log current state
        history["step_index"].append(step)
        history["t_s"].append(t)
        history["h_m"].append(y[0])
        history["V_mps"].append(y[1])
        history["gamma_rad"].append(y[2])
        history["s_m"].append(y[3])

        # stopping conditions
        if y[0] <= min_altitude_m or y[1] <= min_speed_mps:
            break

        # compute derivatives using your math_2d wrapper
        dy = math_2d.rhs_2d(t, y, sigma, vehicle_params, env_params)

        # forward Euler step
        y = [yi + dt_s * dyi for yi, dyi in zip(y, dy)]
        t += dt_s
        step += 1

    # --------------- save to CSVs in data/ ---------------
    save_run(run_config, history)

    # --------------- optional plots ---------------
    if do_plots:
        times = history["t_s"]
        alts  = history["h_m"]
        ranges = history["s_m"]

        plt.figure()
        plt.plot(times, alts)
        plt.xlabel("Time [s]")
        plt.ylabel("Altitude h [m]")
        plt.title(f"2D Planar Re-entry: Altitude vs Time (run {run_id})")
        plt.grid(True)

        plt.figure()
        plt.plot(ranges, alts)
        plt.xlabel("Downrange s [m]")
        plt.ylabel("Altitude h [m]")
        plt.title(f"2D Planar Re-entry Trajectory (run {run_id})")
        plt.grid(True)
        plt.gca().invert_yaxis()
        plt.show()

    return run_id, history


In [13]:
import random

# file to store just the randomized initial states
STATE_CSV = os.path.join(DATA_DIR, "state_csv.csv")

def save_initial_state(run_id: int, init_state: dict) -> None:
    """
    Save one row with the initial state for a given run_id into state_csv.
    Columns: run_id, init_altitude_m, init_speed_mps, init_fpa_deg, sigma_const_deg
    """
    row = {
        "run_id": run_id,
        "init_altitude_m": init_state["init_altitude_m"],
        "init_speed_mps": init_state["init_speed_mps"],
        "init_fpa_deg": init_state["init_fpa_deg"],
        "sigma_const_deg": init_state["sigma_const_deg"],
    }
    df = pd.DataFrame([row])
    if os.path.exists(STATE_CSV):
        df.to_csv(STATE_CSV, mode="a", header=False, index=False)
    else:
        df.to_csv(STATE_CSV, index=False)


# --------- run multiple randomized simulations ---------
N_RUNS = 10  # change as you like

for k in range(N_RUNS):
    # sample a randomized initial state (tweak ranges as you like)
    init_state = {
        # altitude between 80 km and 120 km
        "init_altitude_m": random.uniform(80_000.0, 120_000.0),
        # speed between 7.2 and 7.8 km/s
        "init_speed_mps": random.uniform(7_200.0, 7_800.0),
        # flight-path angle between -10 and -1 degrees
        "init_fpa_deg": random.uniform(-10.0, -1.0),
        # bank between 0 and 70 degrees
        "sigma_const_deg": random.uniform(0.0, 70.0),
    }

    # run sim (no plots for batch mode)
    run_id, _ = run_planar_sim(init_state=init_state, do_plots=False)

    # save the initial state for this run_id
    save_initial_state(run_id, init_state)

print(f"Completed {N_RUNS} randomized runs.")


TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'

In [ ]:
for i in range(10):
    run_planar_sim(do_plots=False)

In [ ]:
def plot_run(run_id: int):
    """Load trajectories for a given run_id from trajectories.csv and plot."""
    traj = pd.read_csv(TRAJ_CSV)
    run_traj = traj[traj["run_id"] == run_id]

    if run_traj.empty:
        print(f"No data found for run_id {run_id}")
        return

    times = run_traj["t_s"].values
    alts  = run_traj["h_m"].values
    ranges = run_traj["s_m"].values

    plt.figure()
    plt.plot(times, alts)
    plt.xlabel("Time [s]")
    plt.ylabel("Altitude h [m]")
    plt.title(f"2D Planar Re-entry: Altitude vs Time (run {run_id})")
    plt.grid(True)

    plt.figure()
    plt.plot(ranges, alts)
    plt.xlabel("Downrange s [m]")
    plt.ylabel("Altitude h [m]")
    plt.title(f"2D Planar Re-entry Trajectory (run {run_id})")
    plt.grid(True)
    plt.gca().invert_yaxis()
    plt.show()


In [ ]:
#plot the data
plot_run(run_id=2)